In [ ]:
import setup

setup.init_django()

In [ ]:
from decouple import config

In [ ]:
from blog.models import BlogPost
from rag import db as rag_db, settings as rag_settings

In [ ]:
from sqlalchemy import (
    create_engine,
    inspect,
)

from llama_index.core import SQLDatabase
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.retrievers import NLSQLRetriever

In [ ]:
# initialize default LlamaIndex settings
rag_settings.init()
# get pooled Neon database string from .env or env vars
vector_database_url = rag_db.get_database_url(use_pooling=True)

In [ ]:
engine = create_engine(vector_database_url)

In [ ]:
inspect(engine).get_table_names()

In [ ]:
tables = []
models = [BlogPost]
for model in models:
    table = model._meta.db_table
    tables.append(table)

In [ ]:
tables

In [ ]:
sql_database = SQLDatabase(engine, include_tables=tables)

In [ ]:
sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=tables,
)

In [ ]:
response = sql_query_engine.query("What is my most recent blog post id?")
response

In [ ]:
for node in response.source_nodes:
    print(node.node.get_content())

In [ ]:
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=tables, return_raw=True
)

r = nl_sql_retriever.retrieve("What is my least most recent blog post?")

In [ ]:
print(r)
for node in r:
    print(node)
    print(node.metadata)